In [1]:
import sys
sys.path.append('../../')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fmdtools.faultprop as fp
import fmdtools.resultproc as rp
import quadpy
from IPython.display import HTML

In [2]:
from fmdtools.modeldef import FxnBlock
from fmdtools.modeldef import Model

### Network Model

In [8]:

class Drone(Model):
    def __init__(self, params={}):
        super().__init__()
        self.params=params
        #add flows to the model
        self.add_flow('Force_ST', 'Force', {'support':1.0})
        self.add_flow('Force_Lin','Force', {'support':1.0} )
        self.add_flow('Force_GR','Force', {'value':1.0} )
        self.add_flow('Force_LG','Force', {'value':1.0})
        self.add_flow('HSig_DOFs','Health Signal', {'hstate':'nominal', 'config':1.0})
        self.add_flow('HSig_Bat','Health Signal', {'hstate':'nominal', 'config':1.0} )
        self.add_flow('RSig_Ctl','Reconfiguration Signal', {'mode':1})
        self.add_flow('RSig_Traj','Reconfiguration Signal', {'mode':1})
        self.add_flow('EE_1', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('EEmot', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('EEctl', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('Ctl1','Direction Signal', {'forward':0.0, 'upward':1.0})
        self.add_flow('DOFs', 'DOFs',{'vertvel':0.0, 'planvel':0.0, 'planpwr':0.0, 'uppwr':0.0})
        self.add_flow('Env1','Environment', {'x':0.0,'y':0.0,'elev':0.0} )
        # custom flows
        self.add_flow('Dir1', 'Direction', {'Dir':1.0})
        #add functions to the model
        flows=['EEctl', 'Force_ST', 'HSig_DOFs', 'HSig_Bat', 'RSig_Ctl', 'RSig_Traj']
        self.add_fxn('StoreEE',Placeholder,['EE_1', 'Force_ST', 'HSig_Bat'])
        self.add_fxn('DistEE',Placeholder, ['EE_1','EEmot','EEctl', 'Force_ST'])
        self.add_fxn('AffectDOF',Placeholder,['EEmot','Ctl1','DOFs','Force_Lin', 'HSig_DOFs'])
        self.add_fxn('CtlDOF', Placeholder,['EEctl', 'Dir1', 'Ctl1', 'DOFs', 'Force_ST', 'RSig_Ctl'])
        self.add_fxn('Planpath', Placeholder, ['EEctl', 'Env1','Dir1', 'Force_ST', 'RSig_Traj'])
        self.add_fxn('Trajectory', Placeholder, ['Env1','DOFs','Dir1', 'Force_GR'] )
        self.add_fxn('EngageLand', Placeholder,['Force_GR', 'Force_LG'])
        self.add_fxn('HoldPayload', Placeholder,['Force_LG', 'Force_Lin', 'Force_ST'])
        self.add_fxn('ViewEnv', Placeholder, ['Env1'])
        
        self.construct_graph()

In [9]:
mdl = Drone()

In [10]:
class Placeholder(FxnBlock):
    def __init__(self, flows):
        super().__init__([str(f) for f in flows], flows)

class Drone(Model):
    def __init__(self, params={}):
        super().__init__()
        self.params=params
        #add flows to the model
        self.add_flow('Force_ST', 'Force', {'support':1.0})
        self.add_flow('Force_Lin','Force', {'support':1.0} )
        self.add_flow('Force_GR','Force', {'value':1.0} )
        self.add_flow('Force_LG','Force', {'value':1.0})
        self.add_flow('HSig_DOFs','Health Signal', {'hstate':'nominal', 'config':1.0})
        self.add_flow('HSig_Bat','Health Signal', {'hstate':'nominal', 'config':1.0} )
        self.add_flow('RSig_Ctl','Reconfiguration Signal', {'mode':1})
        self.add_flow('RSig_Traj','Reconfiguration Signal', {'mode':1})
        self.add_flow('EE_1', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('EEmot', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('EEctl', 'EE', {'rate':1.0, 'effort':1.0})
        self.add_flow('Ctl1','Direction Signal', {'forward':0.0, 'upward':1.0})
        self.add_flow('DOFs', 'DOFs',{'vertvel':0.0, 'planvel':0.0, 'planpwr':0.0, 'uppwr':0.0})
        self.add_flow('Env1','Environment', {'x':0.0,'y':0.0,'elev':0.0} )
        # custom flows
        self.add_flow('Dir1', 'Direction', {'Dir':1.0})
        #add functions to the model
        flows=['EEctl', 'Force_ST', 'HSig_DOFs', 'HSig_Bat', 'RSig_Ctl', 'RSig_Traj']
        self.add_fxn('StoreEE',Placeholder,['EE_1', 'Force_ST', 'HSig_Bat'])
        self.add_fxn('DistEE',Placeholder, ['EE_1','EEmot','EEctl', 'Force_ST'])
        self.add_fxn('AffectDOF',Placeholder,['EEmot','Ctl1','DOFs','Force_Lin', 'HSig_DOFs'])
        self.add_fxn('CtlDOF', Placeholder,['EEctl', 'Dir1', 'Ctl1', 'DOFs', 'Force_ST', 'RSig_Ctl'])
        self.add_fxn('Planpath', Placeholder, ['EEctl', 'Env1','Dir1', 'Force_ST', 'RSig_Traj'])
        self.add_fxn('Trajectory', Placeholder, ['Env1','DOFs','Dir1', 'Force_GR'] )
        self.add_fxn('EngageLand', Placeholder,['Force_GR', 'Force_LG'])
        self.add_fxn('HoldPayload', Placeholder,['Force_LG', 'Force_Lin', 'Force_ST'])
        self.add_fxn('ViewEnv', Placeholder, ['Env1'])
        
        self.construct_graph()

In [11]:
mdl = Drone()

In [14]:
dir(mdl.fxns['StoreEE'])

['<fmdtools.modeldef.Flow object at 0x0000019492290278>',
 '<fmdtools.modeldef.Flow object at 0x0000019492290358>',
 '<fmdtools.modeldef.Flow object at 0x0000019492290400>',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_initstates',
 '_states',
 'add_fault',
 'add_faults',
 'add_he_rate',
 'assoc_modes',
 'behavior',
 'components',
 'condfaults',
 'copy',
 'failrate',
 'faultmodes',
 'faults',
 'flows',
 'has_fault',
 'has_faults',
 'make_flowdict',
 'remove_fault',
 'replace_fault',
 'reset',
 'return_states',
 'time',
 'timely',
 'timers',
 'tstep',
 'type',
 'updatefxn']

In [17]:
dir(mdl.fxns['StoreEE'].flows['<fmdtools.modeldef.Flow object at 0x0000019492290400>'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_initattributes',
 'copy',
 'effort',
 'flow',
 'rate',
 'reset',
 'status',
 'type']

In [18]:
mdl.fxns['StoreEE'].flows['<fmdtools.modeldef.Flow object at 0x0000019492290400>'].type

'flow'

### Static Model

### Dynamic Model

Hierarchical Model